# Deep Research AI Agentic System
This notebook implements a dual-agent system using LangChain and LangGraph frameworks. One agent focuses on research and data collection, while the other drafts answers based on the gathered information.

In [30]:
! pip install langchain-groq


     ---------------------------------------- 0.0/127.4 kB ? eta -:--:--
     ----- ------------------------------- 20.5/127.4 kB 640.0 kB/s eta 0:00:01
     ----- ------------------------------- 20.5/127.4 kB 640.0 kB/s eta 0:00:01
     ----------------- ------------------- 61.4/127.4 kB 544.7 kB/s eta 0:00:01
     ---------------------------- ------- 102.4/127.4 kB 653.6 kB/s eta 0:00:01
     ------------------------------------ 127.4/127.4 kB 680.9 kB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
# Step 1: Install necessary libraries (only if not done)
# pip install langchain langgraph groq requests beautifulsoup4

# Step 2: Imports
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langgraph.graph import StateGraph, END
import requests
from bs4 import BeautifulSoup

# Step 3: Initialize Groq LLM
llm = ChatGroq(
    groq_api_key="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", 
    model_name="llama3-8b-8192",
    temperature=0.2
)

# Step 4: Define basic web crawler with **working URLs**
def simple_web_crawler(query):
    # Replace these with real articles from trusted websites
    return [
        "https://www.nationalgeographic.com/environment/article/solar-energy",
        "https://www.energy.gov/eere/solar/solar-energy-technologies-office",
        "https://www.scientificamerican.com/article/solar-power-advances/"
    ]

def scrape_website(url):
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
        }
        response = requests.get(url, headers=headers, timeout=5)
        soup = BeautifulSoup(response.text, "html.parser")
        text = soup.get_text(separator=' ', strip=True)
        return text[:3000]  # limit text size
    except Exception as e:
        print(f"Failed to scrape {url}: {e}")
        return ""

# Step 5: Define Agents
def research_agent(state):
    query = state["query"]
    links = simple_web_crawler(query)
    gathered_text = []
    for link in links:
        text = scrape_website(link)
        if text:
            gathered_text.append(text)
    return {"query": query, "research_data": gathered_text}

def draft_agent(state):
    research_data = state["research_data"]
    combined_data = "\n".join(research_data)

    prompt = PromptTemplate.from_template(
        """
        Based on the following research data, write a clear, informative, and well-organized answer:

        Research Data:
        {data}

        Final Answer:
        """
    )

    chain = prompt | llm
    output = chain.invoke({"data": combined_data})
    return {"final_answer": output.content}

# Step 6: Define State
class ResearchState(dict):
    query: str
    research_data: list
    final_answer: str

# Step 7: Create LangGraph
graph = StateGraph(ResearchState)
graph.add_node("research_agent", research_agent)
graph.add_node("draft_agent", draft_agent)

graph.set_entry_point("research_agent")
graph.add_edge("research_agent", "draft_agent")
graph.add_edge("draft_agent", END)

compiled_graph = graph.compile()

# Step 8: Run It!
input_query = "Latest advancements in solar panel technology 2025"  # You can change this

result = compiled_graph.invoke({"query": input_query})

print("\n📄 Final Answer:\n")
print(result["final_answer"])



📄 Final Answer:

Based on the research data, it appears that there are three websites: National Geographic, Solar Energy Technologies Office, and Scientific American. Each website provides information on a specific topic.

National Geographic is a website that provides information on various topics such as science, history, and culture. The website also offers maps, books, and other resources for learning.

Solar Energy Technologies Office is a website that provides information on solar energy and its applications. The website offers information on funding opportunities, research areas, and how to apply for funding. The website also provides information on how to save on solar energy and review federal solar tax credit resources.

Scientific American is a website that provides information on science and technology. The website offers news, articles, and other resources on various scientific topics. The website also provides information on subscriptions, institutional access, and more.